# Proceso ETL con Historia
## Francisco Martino, David Romero Acosta, Edgar García
### Modelamiento de Datos y ETL

Se realizó la absorción de los datos desde el origen de archivos `.csv` usando las librerías comunes de Python.

In [1]:
# Importación de librerías
import pandas as pd
import seaborn as sns
import numpy as np

Se llaman los archivos de datos nuevos, proporcionados por el cliente con los que se crearan las dimensiones y tablas de hechos.

In [2]:
aeropuertos =  pd.read_csv("aeropuertos2.csv")
vuelos = pd.read_csv("vuelos2.csv")
cobertura = pd.read_excel("cobertura2.xlsx")

Se eliminan los duplicados en los archivos de origen

In [3]:
aeropuertos.drop_duplicates(inplace=True)
vuelos.drop_duplicates(inplace=True)
cobertura.drop_duplicates(inplace=True)

Se crean las dimensiones y la tabla de hechos

In [4]:
#Dimension Características
caracteristicas = vuelos.loc[:,['tipo_equipo','tipo_vuelo','trafico','empresa']]
caracteristicas.drop_duplicates(inplace=True)
caracteristicas = caracteristicas.sort_values(by = ['tipo_equipo','tipo_vuelo'])
caracteristicas['id'] = np.arange(1,len(caracteristicas)+1, step = 1)
cols=caracteristicas.columns.tolist()
cols = cols[-1:] + cols[:-1]
dim_caracteristicas = caracteristicas[cols]
dim_caracteristicas = dim_caracteristicas.reset_index(drop=True)

#Dimension Tipo_Equipo
tipo_equipo = vuelos.tipo_equipo.unique()
id_tipo_equipo = np.arange(1,len(tipo_equipo)+1, step = 1)
dim_tipo_equipo = pd.DataFrame(data = {'equipo': tipo_equipo,'equipo_key': id_tipo_equipo})

#Dimension Categoria
categoria = aeropuertos.categoria.unique()
id_categoria = np.arange(1,len(categoria)+1, step = 1)
dim_categoria = pd.DataFrame(data = {'id': id_categoria,'categoria': categoria})
dim_categoria = dim_categoria.sort_values(by=['categoria'])

#Dimension Centro Poblado
nombre_cp = cobertura['Nombre centro poblado']
id_cp = cobertura['Codigo Centro Poblado']
dim_centropoblado = pd.DataFrame(data = {'id': id_cp,'nombre': nombre_cp})

#Dimension Fecha
fecha = vuelos.ano
mes = vuelos.mes
dim_fecha = pd.DataFrame(data = {'año': fecha,'mes': mes})
dim_fecha.drop_duplicates(inplace=True)
dim_fecha = dim_fecha.sort_values(by=['año','mes'])
id_fecha = np.arange(1,len(dim_fecha)+1, step = 1)
dim_fecha['id'] = id_fecha
cols=dim_fecha.columns.tolist()
cols = cols[-1:] + cols[:-1]
dim_fecha = dim_fecha[cols]
dim_fecha = dim_fecha.reset_index(drop = True)

#Dimension Geografía

geografia = aeropuertos.loc[:,['gcd_municipio','municipio','gcd_departamento','departamento']]
geografia.drop_duplicates(inplace=True)
geografia = geografia.sort_values(by=['gcd_municipio','departamento','municipio'])
geografia['id'] = np.arange(1,len(geografia)+1, step = 1)
cols=geografia.columns.tolist()
cols = cols[-1:] + cols[:-1]
dim_geografia = geografia[cols]

#Dimensión Aeropuerto
dim_aeropuertos = aeropuertos.loc[:,['sigla','iata','nombre','latitud','longitud','propietario','explotador','longitud_pista','ancho_pista', 'pbmo', 'elevacion', 'resolucion', 'fecha_construccion',
       'fecha_vigencia', 'clase', 'tipo', 'numero_vuelos_origen','gcd_municipio','categoria','Ano']]

dim_aeropuertos['anoInicio'] = dim_aeropuertos.groupby('sigla')['Ano'].shift(1)
dim_aeropuertos.anoInicio.fillna(dim_aeropuertos.Ano, inplace=True)
#dim_aeropuertos.sort_values(by=['sigla','Ano'],ascending=[True, False],inplace = True)
dim_aeropuertos['flagActual'] = dim_aeropuertos.groupby(['sigla'])['Ano'].rank(method='min', ascending = False) 
dim_aeropuertos = dim_aeropuertos.rename(columns={'Ano':'anoFin'})
dim_aeropuertos.flagActual.where(dim_aeropuertos.flagActual == 1, 0, inplace=True)
join_geografia = pd.merge(dim_aeropuertos, dim_geografia, on = 'gcd_municipio', how='left')
join_geografia = join_geografia.loc[:,['sigla','iata','nombre','latitud','longitud','propietario','explotador','longitud_pista','ancho_pista', 'pbmo', 'elevacion', 'resolucion', 'fecha_construccion',
       'fecha_vigencia', 'clase', 'tipo', 'numero_vuelos_origen','categoria','id','anoInicio','anoFin','flagActual']]
join_geografia = join_geografia.rename(columns = {'id':'geografia_id'})
join_categoria = pd.merge(join_geografia, dim_categoria, on = 'categoria', how='left')
join_categoria = join_categoria.drop(columns = ['categoria'])
join_categoria = join_categoria.rename(columns = {'id':'categoria_id'})
dim_aeropuertos = join_categoria.loc[:,['sigla','iata','nombre','latitud','longitud','propietario','explotador','longitud_pista','ancho_pista', 'pbmo', 'elevacion', 'resolucion', 'fecha_construccion',
       'fecha_vigencia', 'clase', 'tipo', 'numero_vuelos_origen','geografia_id','categoria_id','anoInicio','anoFin','flagActual']]
dim_aeropuertos['id'] = np.arange(1,len(join_categoria)+1, step = 1)
cols=dim_aeropuertos.columns.tolist()
cols = cols[-1:] + cols[:-1]
dim_aeropuertos = dim_aeropuertos[cols]
dim_aeropuertos = dim_aeropuertos.reset_index(drop = True)

# Tabla Hechos Cobertura

hechos_cob = cobertura.loc[:,['Tipo_Cobertura','Aeropuerto','Codigo Centro Poblado','Distancia(Km)','Cobertura']]
hechos_cob['id'] = np.arange(1,len(hechos_cob)+1, step = 1)
hechos_cob = hechos_cob.loc[:,['id','Tipo_Cobertura','Aeropuerto','Codigo Centro Poblado','Distancia(Km)','Cobertura']]
hechos_cob = hechos_cob.rename(columns={'Tipo_Cobertura':'TipoCobertura','Aeropuerto':'AeropuertoMasCercano','Cobertura':'flagCobertura','Distancia(Km)':'distancia','Codigo Centro Poblado':'CentroPoblado_id'})

#Tabla de Hechos de Vuelos

#Creacion de auxliar para JOIN de dimension de fecha
aux_dim_fecha = dim_fecha.copy()
aux_dim_fecha['concat_fecha']= aux_dim_fecha['año'].astype('str')+aux_dim_fecha['mes'].astype('str')

#Creacion de auxliar para JOIN de dimension de caracteristicas
aux_dim_caracteristicas = dim_caracteristicas.copy()
aux_dim_caracteristicas['concat_data']= aux_dim_caracteristicas['tipo_equipo'].astype('str')+aux_dim_caracteristicas['tipo_vuelo'].astype('str')+aux_dim_caracteristicas['trafico'].astype('str')+aux_dim_caracteristicas['empresa'].astype('str')

#creacion table de hechos de vuelos
vuelos['concat_fecha'] = vuelos['ano'].astype('str') + vuelos['mes'].astype('str')
fact_vuelos = pd.merge(vuelos,aux_dim_fecha, on = 'concat_fecha', how = 'left')
fact_vuelos = fact_vuelos.drop(columns = ['ano','mes_x','mes_y','concat_fecha','año'])
fact_vuelos.rename(columns={'id':'Fecha_id'},inplace=True)
fact_vuelos['concat_data'] = fact_vuelos['tipo_equipo'].astype('str')+fact_vuelos['tipo_vuelo'].astype('str')+fact_vuelos['trafico'].astype('str')+fact_vuelos['empresa'].astype('str')
fact_vuelos = fact_vuelos.drop(columns = ['tipo_equipo','tipo_vuelo','trafico','empresa'])
fact_vuelos = pd.merge(fact_vuelos,aux_dim_caracteristicas, on = 'concat_data', how = 'left')
fact_vuelos = fact_vuelos.drop(columns = ['concat_data'])
fact_vuelos.rename(columns={'id':'Caracteristicas_id'},inplace=True)
fact_vuelos['id']=np.arange(1,len(fact_vuelos)+1, step = 1)
columnas = ['id','Fecha_id','origen','destino','Caracteristicas_id','vuelos','sillas','carga_ofrecida','pasajeros','carga_bordo']
fact_vuelos = fact_vuelos[columnas]



Se exportan las dimensiones en formato `.csv` para importarlos a la base de datos de SQL Server

In [5]:
dim_caracteristicas.to_csv('dim_caracteristicas.csv', index= False)
dim_tipo_equipo.to_csv('dim_tipo_equipo.csv', index= False)
dim_categoria.to_csv('dim_categoria.csv', index= False)
dim_centropoblado.to_csv('dim_centropoblado.csv', index= False)
dim_fecha.to_csv('dim_fecha.csv', index= False)
dim_geografia.to_csv('dim_geografia.csv', index= False)
dim_aeropuertos.to_csv('dim_aeropuertos.csv', index= False)
hechos_cob.to_csv('hechos_cob.csv', index= False)
fact_vuelos.to_csv('fact_vuelos.csv', index= False)